In [147]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv')
melb_data.head(3)
melb_data.loc[[0, 1, 2],['Address', 'Price']]

,Address,Price
0,85 Turner St,1480000.0
1,25 Bloomburg St,1035000.0
2,5 Charles St,1465000.0


In [148]:
melb_df = melb_data.copy()

In [149]:
melb_df.drop(['index', 'Coordinates'], axis=1, inplace=True)
melb_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,...,2,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019


In [150]:
total_rooms = melb_df['Rooms']+melb_df['Bathroom']+melb_df['Bedroom']
#total_rooms
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
melb_df['MeanRoomsSquare']

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [151]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
melb_df['AreaRatio']

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

In [152]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
sitizen_per_km = countries_df['population']*1e6 / countries_df['square']
print(round(sitizen_per_km.mean(),2))

84.93


In [153]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['Date'].head()
years_sold = melb_df['Date'].dt.year
print(years_sold.head())
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0    2016
1    2016
2    2017
3    2017
4    2016
Name: Date, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [154]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

In [155]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
delta_days.dt.days.head()

0    337
1     34
2    428
3    428
4    155
Name: Date, dtype: int64

In [156]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
print(melb_df['AgeBuilding'].head())
melb_df = melb_df.drop('YearBuilt', axis=1)

0     46
1    116
2    117
3     47
4      2
Name: AgeBuilding, dtype: int64


In [157]:
# practice
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[melb_df['WeekdaySale']>4].shape[0]
print(weekend_count)

12822


In [158]:
print(melb_df['Address'].nunique())
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
print(street_types)

13378
0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object


In [159]:
def change_long(address):
    
    if address == 'Avenue':
        return 'Av'
    elif address == 'Boulevard':
        return 'Bvd'
    elif address == 'Parade':
        return 'Pde'
    else:
        return address

# применить функцию
street_types = street_types.apply(change_long)

In [160]:
print(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: Address, dtype: int64


In [161]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


In [162]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
print(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object


In [163]:
melb_df = melb_df.drop('Address', axis=1)

In [164]:
def get_weekend(weekday):
    
    if weekday > 4:
        return 1
    else:
        return 0


melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(melb_df['Weekend'])

print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 0))

0        1
1        0
2        1
3        1
4        1
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64
1081199.0


In [165]:
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
#print(popular_sellers)
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
print(melb_df['SellerG'].nunique())
print(melb_df['SellerG'].value_counts().head())

50
Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Name: SellerG, dtype: int64


In [166]:
nelson_min_price = melb_df[melb_df['SellerG']=='Nelson']['Price'].min()
other_min_price = melb_df[melb_df['SellerG']=='other']['Price'].min()
print(round(nelson_min_price/other_min_price, 1))

1.3


In [167]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
print(unique_counts)

        Column_Name  Num_Unique            Type
0           Weekend           2           int64
1              Type           3          object
2            Method           5          object
3       WeekdaySale           5           int64
4        Regionname           8          object
5          Bathroom           9           int64
6             Rooms           9           int64
7               Car          11           int64
8        StreetType          11          object
9           Bedroom          12           int64
10        MonthSale          12           int64
11      CouncilArea          33          object
12          SellerG          50          object
13             Date          58  datetime64[ns]
14      AgeBuilding         152           int64
15         Postcode         198           int64
16         Distance         202         float64
17    Propertycount         311           int64
18           Suburb         314          object
19     BuildingArea         602         

In [168]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())
#display(melb_df['Regionname'].cat.codes)

In [169]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})

melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [170]:
old_memory_df = sum(melb_df.memory_usage())/1024/1024
print(old_memory_df)

1.8834762573242188


In [175]:
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
new_memory_df = sum(melb_df.memory_usage())/1024/1024
print(new_memory_df)
print(round(old_memory_df - new_memory_df, 1))
#print(melb_df.info())

1.7977027893066406
0.1
